In [1]:
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm
import scipy.stats as stats
from matplotlib import gridspec
import aplpy as apl #The Astronomy Plotting Library for python
import astropy
from astropy.stats import sigma_clip
from astropy.modeling import functional_models, models, fitting
import astropy.units as U
from astropy.coordinates import ICRS, Galactic, FK4, FK5, Angle, Latitude, Longitude
import astropy.constants as C
from astropy import wcs
import astropy.io.fits as fits
from astropy.io import ascii
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.stats import signal_to_noise_oir_ccd as SNRas
#import spectral_cube as SC
#from spectral_cube import SpectralCube as sc
from astropy.wcs import WCS
matplotlib.use('Agg')
import matplotlib.cm as cm
#import astrometry as ast
import pyfits
import requests
import json
import os
from sklearn import datasets, linear_model
from scipy.optimize import curve_fit
import numpy.ma as ma
from astropy.table import Table, Column
import pyextract
import sewpy
import pandas as pd
from astropy import units as u

/home/lynge/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
sta1_RA_WORLD = SkyCoord('05:16:13.947 -00:09:03.19', unit=(u.hourangle, u.deg)).ra.degree #UCAC3 86-422902
sta1_DEC_WORLD = SkyCoord('05:16:13.947 -00:09:03.19', unit=(u.hourangle, u.deg)).dec.degree

sta2_RA_WORLD = SkyCoord('05:16:26.243 -00:09:04.79', unit=(u.hourangle, u.deg)).ra.degree #TYC 8438-1960-1
sta2_DEC_WORLD = SkyCoord('05:16:26.243 -00:09:04.79', unit=(u.hourangle, u.deg)).dec.degree

sta3_RA_WORLD = SkyCoord('05:16:19.315 -00:13:35.68', unit=(u.hourangle, u.deg)).ra.degree #TYC 8438-1960-1
sta3_DEC_WORLD = SkyCoord('05:16:19.315 -00:13:35.68', unit=(u.hourangle, u.deg)).dec.degree

sky_RA_WORLD = SkyCoord('05:16:17 -00:09:08', unit=(u.hourangle, u.deg)).ra.degree #Random patch of sky
sky_DEC_WORLD = SkyCoord('05:16:17 -00:09:08', unit=(u.hourangle, u.deg)).dec.degree


k_g = 0.16
k_r = 0.08
k_i = 0.02
k_z = 0.01

mag_refsta1_g = 13.291
mag_refsta1_r = 12.843
mag_refsta1_i = 12.668
mag_refsta1_z = 12.593

mag_refsta2_g = 11.689
mag_refsta2_r = 11.491
mag_refsta2_i = 11.465
mag_refsta2_z = 11.500

refsta1_RA = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).ra.degree #P525_E_21_10_17
refsta1_DEC = SkyCoord('00:24:28 07:49:00', unit=(u.hourangle, u.deg)).dec.degree 

refsta2_RA = SkyCoord('00:55:09.9097 +00:43:12.906', unit=(u.hourangle, u.deg)).ra.degree #SA092_342_27_11_17
refsta2_DEC = SkyCoord('00:55:09.9097 +00:43:12.906', unit=(u.hourangle, u.deg)).dec.degree 

In [3]:
QUASAR1_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/g/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/g/')]

QUASAR2_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/g/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/g/')]

refsta1_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/g/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/g/')]

refsta2_g = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/g/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/g/')]

In [4]:
QUASAR1_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/r/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/r/')]

QUASAR2_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/r/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/r/')]

refsta1_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/')]

refsta2_r = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/r/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/r/')]

In [5]:
QUASAR1_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/i/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/i/')]

QUASAR2_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/i/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/i/')]

refsta1_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/')]

refsta2_i = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/i/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/i/')]

In [6]:
QUASAR1_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/z/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/z/')]

QUASAR2_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/z/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/z/')]

refsta1_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/z/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/z/')]

refsta2_z = [os.path.join('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/z/',f) \
        for f in os.listdir('/media/lynge/Lynge-Back-up/Calibration/27-11-17-SA092_342/z/')]

print QUASAR2_z

['/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/z/IMG2017329BLs026.fits', '/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/z/IMG2017329BLs027.fits', '/media/lynge/Lynge-Back-up/Calibration/27-11-17-F51/z/IMG2017329BLs028.fits']


In [7]:
def dist(mid,end):
    return np.sqrt((mid[0] - end[0])**2 + (mid[1] - end[1])**2)

In [8]:
def flux_cal(A_mag,B_mag,B_flux):
    '''Flux of A found through magnitude and flux of B'''
    return B_flux*10**((B_mag-A_mag)/2.5)

In [9]:
def Basic(quasar,header):
    '''Provides a basic list of neccesities from the fits header file for use in the later \
    flux determination, it does not serve an individual purpose beyound that'''
    AGNdata = quasar #np.rot90(np.rot90(np.fliplr(quasar)))
    #header = data[0].header
    #print header
    RAstd = header['CRVAL1']
    DECstd = header['CRVAL2']
    #print RAstd, DECstd
    pixspa = header['PIXSCALE']/(3600.)
    #print pixspa
    RAstdpix = header['CRPIX1']
    DECstdpix = header['CRPIX2']
    exptime = 10 # header['EXPTIME']
    return header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime

In [10]:
def curve(quasar,header,w,center,apparature):
    '''Determines the FLUX of the stellar object given the the numpy array of the image, \
    the header of the fits file, the astropy coordinate representation, \
    the object position and the apparature of interest'''
    header,AGNdata,RAstd,DECstd,pixspa,RAstdpix,DECstdpix,exptime = Basic(quasar,header)
    #AGNdata = np.swapaxes(AGNdata,0,1)
    y, x = np.ogrid[0:header['NAXIS1'],0:header['NAXIS1']]
    #w = np.swapaxes(w,0,1)
    #F_AGN = np.zeros((512,512,3)) #((int(2*apparature/pixspa)+1,int(2*apparature/pixspa)+1,3))
    #print center
    x1 = w.all_world2pix(center[0],center[1],0)
    x1,y1 = (x1[0]),(x1[1])
    try:
        mask = ((y-y1)**2 + (x-x1)**2) > (apparature/float(pixspa))**2
        AGNdata[mask] = float(0)
        mask = AGNdata == 0.
        rows = np.flatnonzero((~mask).sum(axis=1))
        cols = np.flatnonzero((~mask).sum(axis=0))
        AGNdata = AGNdata[rows.min():rows.max()+1, cols.min():cols.max()+1]
        #print apparature/float(pixspa)
    except:
        AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    #if np.sum(AGNdata) < 500:
    #    AGNdata = np.array([[float('nan'),float('nan')],[float('nan'),float('nan')]])
    return AGNdata/float(exptime), x1, y1


In [11]:
def comp1(qua,sta):
    '''Code to determine the relation between the AGN and the Stellar flux as determined by either me \
    or by the SExtractor program respectively'''
    comp = qua/sta
    return comp

In [12]:
def SNR(data1,data2,header,ap1,ap2):
    npix1 = np.pi*ap1**2/((header['PIXSCALE']/3600.)**2)
    npix2 = np.pi*ap2**2/((header['PIXSCALE']/3600.)**2)
    #data2_1 = ((data2-data1)/npix2)*npix1
    data1_2 = data1/10.
    data2_2 = data2/10.
    return SNRas(10.,data1_2,abs(data2_2),0,header['RON'],npix1,gain=header['GAIN'])

In [73]:
def SXtable(quas,aperture):
    header = pyfits.getheader(quas)
    phot_app = aperture*3600/header['PIXSCALE']
    np.set_printoptions(threshold=np.inf)
    sew = sewpy.SEW(params=['XMODEL_WORLD', 'YMODEL_WORLD', 'X2MODEL_WORLD', 'Y2MODEL_WORLD', 
                            'XMODEL_IMAGE', 'YMODEL_IMAGE', 'FLUX_RADIUS(1)', 'FLUX_APER','ELLIPTICITY'],
                    config={"DETECT_THRESH":5, 'DETECT_MINAREA':20, 'DETECT_MAXAREA':30000, 
                            "PHOT_FLUXFRAC":"0.5", 'PHOT_APERTURES':2*phot_app, 
                            'PIXEL_SCALE':header['PIXSCALE']}
                    , sexpath='/usr/bin/sextractor')
    out = sew(quas)
    return out

In [74]:
def flux(quas,aperture,main_RA_WORLD,main_DEC_WORLD):
    header = pyfits.getheader(quas)
    RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[0]
    DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,1)[1]
    output = float('nan')
    if 15 < RAmain < header['NAXIS1'] - 15 and 15 < DECmain < header['NAXIS2'] - 15: 
        out = SXtable(quas,aperture)
        #print out['table']['XMODEL_IMAGE']
        #print RAmain
        RAmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[0]
        DECmain = WCS(quas).all_world2pix(main_RA_WORLD,main_DEC_WORLD,0)[1]
        #print out['table']['XMODEL_IMAGE'], out['table']['YMODEL_IMAGE']
        #print RAmain, DECmain

        for j in range(len(out['table']['XMODEL_IMAGE'])):
            #output['time'][i] = header['MJD-OBS']
            if RAmain - 15. < out['table']['XMODEL_IMAGE'][j] <  RAmain + 15. and \
            DECmain - 15. < out['table']['YMODEL_IMAGE'][j] < DECmain + 15.:
                #print 'YES'
                output = out['table']['FLUX_APER'][j]
                break
    else:
        output = float('nan')
        #print 'Done'
    return output


In [76]:
def findmag(start,compare1,RA_star,DEC_star,RA_com,DEC_com,mag_com,AIR_EXTINCTION):
    print mag_com
    aperture1 = 0.001
    #201.5441,-8.8601
    com_flu = []
    com_AIRMASS = []
    com_EXPTIME = []
    star_flu = []
    star_AIRMASS = []
    star_EXPTIME = []
    for i in range(len(compare1)):
        print 'Frame nr. ', i, compare1[i]
        header = pyfits.getheader(compare1[i])
        a1 = flux(compare1[i],2*aperture1,RA_com,DEC_com)/float(header['EXPTIME'])
        print header['EXPTIME']
        if a1 > 0:
            #print a1
            com_flu.append(a1), com_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), com_EXPTIME.append(header['EXPTIME'])
        else:
            print a1
            com_flu.append(float('nan')), com_AIRMASS.append(float('nan')), com_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    for i in range(len(start)):
        print 'Frame nr. ', i, start[i]
        header = pyfits.getheader(start[i])
        a1 = flux(start[i],2*aperture1,RA_star,DEC_star)/float(header['EXPTIME'])
        print header['EXPTIME']
        if a1 > 0:
            #print a1
            star_flu.append(a1), star_AIRMASS.append(header['AIRMASS']*AIR_EXTINCTION), star_EXPTIME.append(header['EXPTIME'])
        else:
            star_flu.append(float('nan')), star_AIRMASS.append(float('nan')), star_EXPTIME.append(header['EXPTIME'])
        #plt.figure(figsize=(3,3))
        #plt.imshow(a1,cmap='hot')
        #plt.colorbar()
    com_flu = np.array(com_flu)
    print com_flu
    com_AIRMASS = np.array(com_AIRMASS)
    com_EXPTIME = np.array(com_EXPTIME)
    star_flu = np.array(star_flu)
    print star_flu
    star_AIRMASS = np.array(star_AIRMASS)
    star_EXPTIME = np.array(star_EXPTIME)
    com_flu = com_flu#/com_EXPTIME
    star_flu = star_flu#/star_EXPTIME
    com_flux = np.nanmean(com_flu)
    com_AIRMASS_num = np.nanmean(com_AIRMASS)
    star_AIRMASS_num = np.nanmean(star_AIRMASS)
    print star_AIRMASS_num
    mag1 = (star_AIRMASS_num-com_AIRMASS_num) + mag_com - 2.5*np.log10(star_flu/com_flux)
    mag = mag1
    print (star_AIRMASS_num-com_AIRMASS_num)
    print mag_com, com_flux, np.average(com_flu)
    print mag
    return mag

In [67]:
com1_1 = findmag(QUASAR1_g,refsta1_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com2_1 = findmag(QUASAR1_g,refsta1_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)
com3_1 = findmag(QUASAR1_g,refsta1_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_g,k_g)

#com1_2 = findmag(QUASAR2_g,refsta2_g,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
#com2_2 = findmag(QUASAR2_g,refsta2_g,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)
#com3_2 = findmag(QUASAR2_g,refsta2_g,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_g,k_g)

13.291
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/g/IMG2017292URh017.fits
INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the C

KeyboardInterrupt: 

In [23]:
print 'g-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

g-filter
Star 1
[ 13.4837997   13.48199172  13.48566875]

NameError: name 'com1_2' is not defined

In [77]:
com1_1 = findmag(QUASAR1_r,refsta1_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com2_1 = findmag(QUASAR1_r,refsta1_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)
com3_1 = findmag(QUASAR1_r,refsta1_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_r,k_r)

#com1_2 = findmag(QUASAR2_r,refsta2_r,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
#com2_2 = findmag(QUASAR2_r,refsta2_r,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)
#com3_2 = findmag(QUASAR2_r,refsta2_r,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_r,k_r)

12.843
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRh017.fits
40.0
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRh018.fits
40.0
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRh021.fits
40.0
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRh022.fits
40.0
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRs019.fits
20.0
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRs020.fits
20.0
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRs023.fits
20.0
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/r/IMG2017292BRs024.fits
20.0
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/r/IMG2017292BRs141.fits
45.0
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/r/IMG2017292BRs142.fits
45.0
Frame nr.  2 /media/lyn

In [20]:
print 'r-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

r-filter
Star 1
[ 12.81060995  12.80931484  12.80524056]

NameError: name 'com1_2' is not defined

In [78]:
com1_1 = findmag(QUASAR1_i,refsta1_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com2_1 = findmag(QUASAR1_i,refsta1_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)
com3_1 = findmag(QUASAR1_i,refsta1_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_i,k_i)

#com1_2 = findmag(QUASAR2_i,refsta2_i,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
#com2_2 = findmag(QUASAR2_i,refsta2_i,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)
#com3_2 = findmag(QUASAR2_i,refsta2_i,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_i,k_i)

12.668
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULh017.fits
40.0
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULh018.fits
40.0
Frame nr.  2 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULh021.fits
40.0
Frame nr.  3 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULh022.fits
40.0
Frame nr.  4 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULs019.fits
20.0
Frame nr.  5 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULs020.fits
20.0
Frame nr.  6 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULs023.fits
20.0
Frame nr.  7 /media/lynge/Lynge-Back-up/Calibration/21-10-17-P525-E/i/IMG2017292ULs024.fits
20.0
Frame nr.  0 /media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/i/IMG2017292ULs141.fits
45.0
Frame nr.  1 /media/lynge/Lynge-Back-up/Calibration/21-10-17-AKN120/i/IMG2017292ULs142.fits
45.0
Frame nr.  2 /media/lyn

In [ ]:
print 'i-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

In [ ]:
com1_1 = findmag(QUASAR1_z,refsta1_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com2_1 = findmag(QUASAR1_z,refsta1_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)
com3_1 = findmag(QUASAR1_z,refsta1_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta1_RA,refsta1_DEC,mag_refsta1_z,k_z)

#com1_2 = findmag(QUASAR2_z,refsta2_z,sta1_RA_WORLD,sta1_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
#com2_2 = findmag(QUASAR2_z,refsta2_z,sta2_RA_WORLD,sta2_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)
#com3_2 = findmag(QUASAR2_z,refsta2_z,sta3_RA_WORLD,sta3_DEC_WORLD,refsta2_RA,refsta2_DEC,mag_refsta2_z,k_z)

In [ ]:
print 'z-filter'
print 'Star 1'
print com1_1,com1_2
print np.nanmean((com1_1,com1_2)), np.nanstd((com1_1,com1_2))

print 'Star 2'
print com2_1,com2_2
print np.nanmean((com2_1,com2_2)), np.nanstd((com2_1,com2_2))

print 'Star 3'
print com3_1,com3_2
print np.nanmean((com3_1,com3_2)), np.nanstd((com3_1,com3_2))

In [39]:
print np.log10(3/4.)


-0.124938736608
